## 테스트 환경
이 글의 모든 작업은 fedora 38에서 실행확인되었다.
사용언어는 C




## 타겟 플랫폼 설정

윈도우,리눅스

### 문제점
터미널 텍스트기반 그래픽을사용하면
윈도우 터미널과 리눅스 터미널의 명령어 체계가 달라서 각 플랫폼마다 분기를 설정해 컴파일 해야된다는 불편한 점이있다.
또 본인은 리눅스가 메인 운영체제이기떄문에 윈도우에서 재대로 돌아가는지 직접 확인하려면 거추장스러운 일련의 과정을 거쳐야한다.


#### 해결방법
크로스플랫폼지원 그래픽 인터페이스를 사용한다.
SDL을 사용하는것이 적절하겠다.






## 필요 라이브러리 설치

```bash
sudo dnf update -y
sudo dnf install SDL2-devel -y
sudo dnf install gcc -y
sudo dnf install SDL2_ttf-devel -y

```

### 링크 방법

파일이름을 myprogram로 가정한다.

gcc에서 컴파일 할떄 설치한 라이브러리를 링크하려면
```bash
gcc -o myprogram myprogram.c -lSDL2 -lSDL2_ttf
```


### 폰트 파일 다운로드
https://www.dafont.com/tetris.font





## 구현

SDL을 초기화하고, 블록들이 떨어질 프레임을 출력해보자.

### SDL초기화

```c
#include <SDL2/SDL.h>

int main() {
    SDL_Window *window = NULL;
    SDL_Surface *screenSurface = NULL;

    // SDL 초기화
    if (SDL_Init(SDL_INIT_VIDEO) < 0) {
        printf("SDL 초기화 실패: %s\n", SDL_GetError());
        return -1;
    }

    // 게임 윈도우 생성
    window = SDL_CreateWindow("My Game", SDL_WINDOWPOS_UNDEFINED, SDL_WINDOWPOS_UNDEFINED,
                              640, 480, SDL_WINDOW_SHOWN);
    if (window == NULL) {
        printf("윈도우 생성 실패: %s\n", SDL_GetError());
        return -1;
    }

    // 윈도우 표면 가져오기
    screenSurface = SDL_GetWindowSurface(window);

    // 윈도우 배경색 설정
    SDL_FillRect(screenSurface, NULL, SDL_MapRGB(screenSurface->format, 0, 0, 0));

    // 배경색 표시
    SDL_UpdateWindowSurface(window);

    // 대기
    SDL_Delay(5000);

    // 리소스 해제
    SDL_DestroyWindow(window);
    SDL_Quit();

    return 0;
}
```

### 테트리스 프레임 출력

테트리스 프레임임 크기의 비율은 10 * 20 로 설정하고.
프레임을 출력해보자.


```c
#include <SDL2/SDL.h>

int main() {
    SDL_Window *window = NULL;
    SDL_Surface *screenSurface = NULL;

    // SDL 초기화
    if (SDL_Init(SDL_INIT_VIDEO) < 0) {
        printf("SDL 초기화 실패: %s\n", SDL_GetError());
        return -1;
    }

    // 게임 윈도우 생성
    window = SDL_CreateWindow("My Game", SDL_WINDOWPOS_UNDEFINED, SDL_WINDOWPOS_UNDEFINED,
                              640, 480, SDL_WINDOW_SHOWN);
    if (window == NULL) {
        printf("윈도우 생성 실패: %s\n", SDL_GetError());
        return -1;
    }

    // 윈도우 표면 가져오기
    screenSurface = SDL_GetWindowSurface(window);

    // 윈도우 배경색 설정
    SDL_FillRect(screenSurface, NULL, SDL_MapRGB(screenSurface->format, 0, 0, 0));

    // 빨간색 가로선과 세로선 그리기
    SDL_Rect lineRect;
    lineRect.x = 160;
    lineRect.y = 430;
    lineRect.w = 320;
    lineRect.h = 1;
    SDL_FillRect(screenSurface, &lineRect, SDL_MapRGB(screenSurface->format, 255, 0, 0));

    lineRect.x = 160;
    lineRect.y = 46;
    lineRect.w = 1;
    lineRect.h = 384;
    SDL_FillRect(screenSurface, &lineRect, SDL_MapRGB(screenSurface->format, 255, 0, 0));

    lineRect.x = 480;
    SDL_FillRect(screenSurface, &lineRect, SDL_MapRGB(screenSurface->format, 255, 0, 0));

    // 화면 업데이트
    SDL_UpdateWindowSurface(window);

    // 대기
    SDL_Delay(5000);

    // 리소스 해제
    SDL_DestroyWindow(window);
    SDL_Quit();

    return 0;
}
```

#### 문제점

##### 코드반복
코드반복부분을 수정해보자
```c
// 빨간색 가로선과 세로선 그리기
    SDL_Rect lineRect;
    lineRect.x = 160;
    lineRect.y = 430;
    lineRect.w = 320;
    lineRect.h = 1;
    SDL_FillRect(screenSurface, &lineRect, SDL_MapRGB(screenSurface->format, 255, 0, 0));

    lineRect.x = 160;
    lineRect.y = 46;
    lineRect.w = 1;
    lineRect.h = 384;
    SDL_FillRect(screenSurface, &lineRect, SDL_MapRGB(screenSurface->format, 255, 0, 0));

    lineRect.x = 480;
    SDL_FillRect(screenSurface, &lineRect, SDL_MapRGB(screenSurface->format, 255, 0, 0));
```
반복문을 이용해 수정해보자.

```c
SDL_Rect lineRects[3] = {
    {160, 430, 320, 1},
    {160, 46, 1, 384},
    {480, 46, 1, 384}
};

for (int i = 0; i < 3; i++) {
    SDL_FillRect(screenSurface, &lineRects[i], SDL_MapRGB(screenSurface->format, 255, 0, 0));
}

````

##### 어떤 색상을 사용하는지 알기 어려움
색상값을 구조체와 enum을 이용해 가독성을 높여보자
```c
typedef struct {
    Uint8 r;
    Uint8 g;
    Uint8 b;
} Color;

enum {
    BLACK = 0,
    WHITE = 1,
    RED = 2,
    GREEN = 3,
    BLUE = 4,
    YELLOW = 5,
    MAGENTA = 6,
    CYAN = 7,
    GRAY = 8,
    DARK_GRAY = 9,
    LIGHT_GRAY = 10
};

const Color colors[] = {
    {0, 0, 0},          // BLACK
    {255, 255, 255},    // WHITE
    {255, 0, 0},        // RED
    {0, 255, 0},        // GREEN
    {0, 0, 255},        // BLUE
    {255, 255, 0},      // YELLOW
    {255, 0, 255},      // MAGENTA
    {0, 255, 255},      // CYAN
    {128, 128, 128},    // GRAY
    {64, 64, 64},       // DARK_GRAY
    {192, 192, 192}     // LIGHT_GRAY
};

...
SDL_FillRect(screenSurface, &lineRects[i], SDL_MapRGB(screenSurface->format, colors[RED].r, colors[RED].g, colors[RED].b));
```

#### 수정된 코드
```c
#include <SDL2/SDL.h>

typedef struct {
    Uint8 r;
    Uint8 g;
    Uint8 b;
} Color;

enum {
    BLACK = 0,
    WHITE = 1,
    RED = 2,
    GREEN = 3,
    BLUE = 4,
    YELLOW = 5,
    MAGENTA = 6,
    CYAN = 7,
    GRAY = 8,
    DARK_GRAY = 9,
    LIGHT_GRAY = 10
};

const Color colors[] = {
    {0, 0, 0},          // BLACK
    {255, 255, 255},    // WHITE
    {255, 0, 0},        // RED
    {0, 255, 0},        // GREEN
    {0, 0, 255},        // BLUE
    {255, 255, 0},      // YELLOW
    {255, 0, 255},      // MAGENTA
    {0, 255, 255},      // CYAN
    {128, 128, 128},    // GRAY
    {64, 64, 64},       // DARK_GRAY
    {192, 192, 192}     // LIGHT_GRAY
};

int main() {
    SDL_Window *window = NULL;
    SDL_Surface *screenSurface = NULL;

    // SDL 초기화
    if (SDL_Init(SDL_INIT_VIDEO) < 0) {
        printf("SDL 초기화 실패: %s\n", SDL_GetError());
        return -1;
    }

    // 게임 윈도우 생성
    window = SDL_CreateWindow("My Game", SDL_WINDOWPOS_UNDEFINED, SDL_WINDOWPOS_UNDEFINED,
                              640, 480, SDL_WINDOW_SHOWN);
    if (window == NULL) {
        printf("윈도우 생성 실패: %s\n", SDL_GetError());
        return -1;
    }

    // 윈도우 표면 가져오기
    screenSurface = SDL_GetWindowSurface(window);

    // 윈도우 배경색 설정
    SDL_FillRect(screenSurface, NULL, SDL_MapRGB(screenSurface->format, colors[BLACK].r, colors[BLACK].g, colors[BLACK].b));

    // 빨간색 가로선과 세로선 그리기
    SDL_Rect lineRects[3] = {
        {160, 430, 320, 1},
        {160, 46, 1, 384},
        {480, 46, 1, 384}
    };

    for (int i = 0; i < 3; i++) {
        SDL_FillRect(screenSurface, &lineRects[i], SDL_MapRGB(screenSurface->format, colors[RED].r, colors[RED].g, colors[RED].b));}

    // 화면 업데이트
    SDL_UpdateWindowSurface(window);

    // 대기
    SDL_Delay(5000);

    // 리소스 해제
    SDL_DestroyWindow(window);
    SDL_Quit();

    return 0;
}

```

### 블록생성, 출력

#### 블록생성

블록의 외곽선을 흰색으로 설정해서 개별블록마다 인식성을 높였고,
사용할 색상을 정해놓고 색상값과 위치, 뿌려줄 렌더러를 넘기는 함수로 묶었다.

```c
void Draw_rect(SDL_Renderer* renderer, Color color, int x, int y) {
    SDL_SetRenderDrawColor(renderer, 255, 255, 255, 255);
    SDL_RenderDrawRect(renderer, &(SDL_Rect){ x, y, RECTANGLE_SIZE, RECTANGLE_SIZE });

    switch (color) {
        case BLACK:
            SDL_SetRenderDrawColor(renderer, 100, 100, 100, 255);
            break;
        case WHITE:
            SDL_SetRenderDrawColor(renderer, 100, 100, 100, 255);
            break;
        case RED:
            SDL_SetRenderDrawColor(renderer, 255, 0, 0, 255);
            break;
        case GREEN:
            SDL_SetRenderDrawColor(renderer, 0, 255, 0, 255);
            break;
        case BLUE:
            SDL_SetRenderDrawColor(renderer, 0, 0, 255, 255);
            break;
        case YELLOW:
            SDL_SetRenderDrawColor(renderer, 255, 255, 0, 255);
            break;
        case MAGENTA:
            SDL_SetRenderDrawColor(renderer, 255, 0, 255, 255);
            break;
        case CYAN:
            SDL_SetRenderDrawColor(renderer, 0, 255, 255, 255);
            break;
        case GRAY:
            SDL_SetRenderDrawColor(renderer, 128, 128, 128, 255);
            break;
        default:
            break;
    }

    SDL_RenderFillRect(renderer, &(SDL_Rect){ x + 1, y + 1, 18, 18 });
}

```



#### 문제점
##### 기본 블럭크기를 설정하지 않았다.
1블록의 크기를 지정한후 프레임을 크기를 지정해야 빈공간없이 화면을 채울수 있다.


테트리스 프레임 비율을 10 * 20으로 설정한다면

블록으로 빈공간없이 정렬상태의 화면을 유지하려면

640 * 480의 화면크기에서 20 * 20 크기의 블록을 설정하면된다는 결론이 나온다.


##### 화면 출력방식
블록단위로 화면을 출력할 예정이니 


화면상태를 2차원리스트로 설정하고,


해당 위치의 값을 참고해서 블록의 종류를 선택하고 한번에 그리는게 효율적일거같다.


640 * 480 화면이니 Uint8 matrix[32][24] 로 선언

아래는 프레임을 그리는 코드를 재작성한것이다.

MATRIX_WIDTH  == 32

MATRIX_HEIGHT == 24

if문은 프레임 위치의 x,y값을 설정해 둔것이고, 사각형 그리는함수에 *20 을 한 이유는 사각형크기가 20 * 20이기 때문이다. 

```c
    // 행렬 초기화
    uint8_t matrix[MATRIX_WIDTH][MATRIX_HEIGHT];
    for (int x = 0; x < MATRIX_WIDTH; x++) {
        for (int y = 0; y < MATRIX_HEIGHT; y++) {
            if ((((x-10)==0 || (x-21)==0) && ((y-2>=0)&&(y<=21)) || (((x-10>0) && (x-21<0)) && (y-21==0)) ) ){
		    Draw_rect(renderer,GRAY,x*20,y*20);
            }
       }
       }
```

##### 색상구조체

색상구조체를 사용하니 오히려 더 불편했다.

rgba값을 직접 입력하고, 함수 호출 부분에서만 enum을 사용하게 코드를 변경했다.


#### 수정된 코드
```c
#include <SDL2/SDL.h>

#define RECTANGLE_SIZE 20

#define MATRIX_WIDTH 32
#define MATRIX_HEIGHT 24

typedef enum {
    BLACK = 0,
    WHITE = 1,
    RED = 2,
    GREEN = 3,
    BLUE = 4,
    YELLOW = 5,
    MAGENTA = 6,
    CYAN = 7,
    GRAY = 8,
} Color;

void Draw_rect(SDL_Renderer* renderer, Color color, int x, int y);

int main(int argc, char* argv[])
{
    // SDL 초기화
    if (SDL_Init(SDL_INIT_VIDEO) != 0) {
        SDL_Log("SDL_Init failed: %s", SDL_GetError());
        return 1;
    }

    // 윈도우 생성
    SDL_Window* window = SDL_CreateWindow("SDL Example",
        SDL_WINDOWPOS_UNDEFINED,
        SDL_WINDOWPOS_UNDEFINED,
        640, 480,
        SDL_WINDOW_SHOWN);
    if (window == NULL) {
        SDL_Log("SDL_CreateWindow failed: %s", SDL_GetError());
        return 1;
    }

    // 렌더러 생성
    SDL_Renderer* renderer = SDL_CreateRenderer(window, -1, SDL_RENDERER_ACCELERATED);
    if (renderer == NULL) {
        SDL_Log("SDL_CreateRenderer failed: %s", SDL_GetError());
        return 1;
    }

    // 행렬 초기화
    // 0 == 빈공간 , 1 프레임  , 2~ 테트리스블록 
    uint8_t matrix[MATRIX_WIDTH][MATRIX_HEIGHT];
    for (int x = 0; x < MATRIX_WIDTH; x++) {
        for (int y = 0; y < MATRIX_HEIGHT; y++) {
            if ((((x-10)==0 || (x-21)==0) && ((y-2>=0)&&(y<=21)) || (((x-10>0) && (x-21<0)) && (y-21==0)) ) ){
		    matrix[x][y] = 1;
	    }
    }
    }


    for (int x = 0; x < MATRIX_WIDTH; x++) {
        for (int y = 0; y < MATRIX_HEIGHT; y++) {
		if(matrix[x][y]==1){
			Draw_rect(renderer,GRAY,x*20,y*20);
		}
	}
    }
    


    // 화면 업데이트
    SDL_RenderPresent(renderer);

    // 이벤트 처리
    while (1) {
        SDL_Event e;
        if (SDL_PollEvent(&e) && e.type == SDL_QUIT) {
            break;
        }
    }

    // 메모리 해제
    SDL_DestroyRenderer(renderer);
    SDL_DestroyWindow(window);
    SDL_Quit();

    return 0;
}

void Draw_rect(SDL_Renderer* renderer, Color color, int x, int y) {
    SDL_SetRenderDrawColor(renderer, 255, 255, 255, 255);
    SDL_RenderDrawRect(renderer, &(SDL_Rect){ x, y, RECTANGLE_SIZE, RECTANGLE_SIZE });

    switch (color) {
        case BLACK:
            SDL_SetRenderDrawColor(renderer, 100, 100, 100, 255);
            break;
        case WHITE:
            SDL_SetRenderDrawColor(renderer, 100, 100, 100, 255);
            break;
        case RED:
            SDL_SetRenderDrawColor(renderer, 255, 0, 0, 255);
            break;
        case GREEN:
            SDL_SetRenderDrawColor(renderer, 0, 255, 0, 255);
            break;
        case BLUE:
            SDL_SetRenderDrawColor(renderer, 0, 0, 255, 255);
            break;
        case YELLOW:
            SDL_SetRenderDrawColor(renderer, 255, 255, 0, 255);
            break;
        case MAGENTA:
            SDL_SetRenderDrawColor(renderer, 255, 0, 255, 255);
            break;
        case CYAN:
            SDL_SetRenderDrawColor(renderer, 0, 255, 255, 255);
            break;
        case GRAY:
            SDL_SetRenderDrawColor(renderer, 128, 128, 128, 255);
            break;
        default:
            break;
    }

    SDL_RenderFillRect(renderer, &(SDL_Rect){ x + 1, y + 1, 18, 18 });
}



```

### 테트리스 블록 생성
테트리스는 7가지 모양의 블록이있다. 각 모양마다 숫자를 부여하고

그 숫자에 해당하는 색상의 블록으로 렌더링하면 될거같다.

enum CellStatus 값을 활용해주기위해 block 리스트의 0~1은 빈값으로 채워넣었다.

```c

typedef enum {
    EMPTY,
    FRAME,
    BLOCK_SQUARE,
    BLOCK_LINE,
    BLOCK_T,
    BLOCK_L,
    BLOCK_L_REV,
    BLOCK_ZIGZAG,
    BLOCK_ZIGZAG_REV
} CellStatus;

typedef enum {
    BLACK ,
    WHITE ,
    RED ,
    GREEN,
    BLUE ,
    YELLOW ,
    MAGENTA ,
    CYAN , 
    GRAY ,
    ORANGE,
    PURPLE 
} Color;


uint8_t blocks[9][2][4] = {
	{//EMPTY :0
		{0,0,0,0},
		{0,0,0,0}
	},
	{//FRAME :1
		{0,0,0,0},
		{0,0,0,0}
	},
	{//네모 :2
		{2,2,0,0},
		{2,2,0,0}
	},

	{//직선 : 3
		{3,3,3,3},
		{0,0,0,0}
	},
	{// T : 4  
	
		{4,4,4,0},
		{0,4,0,0}
	},
	{// L  :5 
		{0,0,5,0},
		{5,5,5,0}
	},
	{ //반대 L :6
		{6,0,0,0},
		{6,6,6,0}
	},
	{ //지그제그  : 7
		{0,7,7,0},
		{7,7,0,0}
	},
	{ //반대 지그제그  : 8
		{8,8,0,0},
		{0,8,8,0}
	}

};


```

#### 초기 블록 생성
현재블록과 다음에 생성될 블록을 표현

##### 폰트 설정
게임에 출력될 텍스트를 설정해보자.
SDL은 폰트파일에서 값을 읽어 화면에 뿌려주는 함수가 존재한다. 그함수를 이용한다.


##### 문제점
2차원 리스트의 xy위치를 잘못설정했다.
matrix[height][width] 형식으로 접근해야되는 것이맞다.


##### 수정된 코드
```c
#include <SDL2/SDL.h>
#include <SDL2/SDL_ttf.h>
#include <time.h>

#define BLOCK_SIZE 20

#define MATRIX_WIDTH 32
#define MATRIX_HEIGHT 24

typedef enum {
    EMPTY,
    FRAME,
    BLOCK_SQUARE,
    BLOCK_LINE,
    BLOCK_T,
    BLOCK_L,
    BLOCK_L_REV,
    BLOCK_ZIGZAG,
    BLOCK_ZIGZAG_REV
} CellStatus;

typedef enum {
    BLACK ,
    WHITE ,
    RED ,
    GREEN,
    BLUE ,
    YELLOW ,
    MAGENTA ,
    CYAN , 
    GRAY ,
    ORANGE,
    PURPLE 
} Color;


uint8_t blocks[9][2][4] = {
	{//EMPTY :0
		{0,0,0,0},
		{0,0,0,0}
	},
	{//FRAME :1
		{0,0,0,0},
		{0,0,0,0}
	},
	{//네모 :2
		{2,2,0,0},
		{2,2,0,0}
	},

	{//직선 : 3
		{3,3,3,3},
		{0,0,0,0}
	},
	{// T : 4  
	
		{4,4,4,0},
		{0,4,0,0}
	},
	{// L  :5 
		{0,0,5,0},
		{5,5,5,0}
	},
	{ //반대 L :6
		{6,0,0,0},
		{6,6,6,0}
	},
	{ //지그제그  : 7
		{0,7,7,0},
		{7,7,0,0}
	},
	{ //반대 지그제그  : 8
		{8,8,0,0},
		{0,8,8,0}
	}

};


void Draw_rect(Color color, int x, int y);
void Spawn_block();
void draw_text(int x,int y,char* str,SDL_Color textColor);
void draw_game_board();


CellStatus matrix[MATRIX_HEIGHT][MATRIX_WIDTH];
TTF_Font *font;
SDL_Renderer* renderer; 
int main(int argc, char* argv[])
{
	SDL_Color textColor = { 255, 255, 255,255 }; // 빨간색

	
    // SDL 초기화
    if (SDL_Init(SDL_INIT_VIDEO) != 0) {
        SDL_Log("SDL_Init failed: %s", SDL_GetError());
        return 1;
    }
     // TTF 초기화
    if (TTF_Init() != 0) {
        SDL_Log("TTF_Init failed: %s", TTF_GetError());
        return 1;
    }

    // 윈도우 생성
    SDL_Window* window = SDL_CreateWindow("SDL Example",
        SDL_WINDOWPOS_UNDEFINED,
        SDL_WINDOWPOS_UNDEFINED,
        640, 480,
        SDL_WINDOW_SHOWN);
    if (window == NULL) {
        SDL_Log("SDL_CreateWindow failed: %s", SDL_GetError());
        return 1;
    }

    // 렌더러 생성
    renderer = SDL_CreateRenderer(window, -1, SDL_RENDERER_ACCELERATED);
    if (renderer == NULL) {
        SDL_Log("SDL_CreateRenderer failed: %s", SDL_GetError());
        return 1;
    }

    // 폰트 로드
    
    font = TTF_OpenFont("Tetris.ttf",28);
    if (font==NULL){
	    SDL_Log("TTF_OpenFont failed: %s",TTF_GetError());
	    return 1;
    }
    

    for (int x = 0; x < MATRIX_WIDTH; x++) {
        for (int y = 0; y < MATRIX_HEIGHT; y++) {
            if ((((x-10)==0 || (x-21)==0) && ((y-2>=0)&&(y<=21)) || (((x-10>0) && (x-21<0)) && (y-21==0)) ) ){
		    matrix[y][x] = FRAME;
	    }
    }
    }
    SDL_Color white = {255,255,255,255};
    SDL_Color yellow = {255, 255, 0, 255};
    SDL_Color orange = {255, 165, 0, 255};
    SDL_Color red = {255, 0, 0, 255};

    draw_text(40,40,"NEXT",red);
    draw_text(480,40,"SCORE",white);
    draw_text(480,70,"0",yellow);

    draw_text(480,130,"LINES",white);
    draw_text(480,160,"0",orange);

    draw_text(480,220,"LEVEL",white);
    draw_text(480,250,"0",red);
    Spawn_block();
    draw_game_board(renderer);

    // 화면 업데이트
    SDL_RenderPresent(renderer);

// 이벤트 처리
while (1) {
    SDL_Event e;
    if (SDL_PollEvent(&e)) {
        if (e.type == SDL_QUIT) {
            break;
        } else if (e.type == SDL_KEYDOWN) {
            switch (e.key.keysym.sym) {
                case SDLK_SPACE:
                    // 스페이스바 눌림
                    break;
                case SDLK_LEFT:
                    // 왼쪽 방향키 눌림
                    break;
                case SDLK_RIGHT:
                    // 오른쪽 방향키 눌림
                    break;
                case SDLK_UP:
                    // 위쪽 방향키 눌림
                    break;
                case SDLK_DOWN:
                    // 아래쪽 방향키 눌림
                    break;
            }
        }
    }
}

    // 메모리 해제
    SDL_DestroyRenderer(renderer);
    SDL_DestroyWindow(window);
    SDL_Quit();

    return 0;
}

void Draw_rect(Color color, int y, int x) {
    SDL_SetRenderDrawColor(renderer, 255, 255, 255, 255);
    SDL_RenderDrawRect(renderer, &(SDL_Rect){ x, y, BLOCK_SIZE, BLOCK_SIZE });

    switch (color) {
    case BLACK:
        SDL_SetRenderDrawColor(renderer, 0x64, 0x64, 0x64, 0xFF);
        break;
    case WHITE:
        SDL_SetRenderDrawColor(renderer, 0xFF, 0xFF, 0xFF, 0xFF);
        break;
    case RED:
        SDL_SetRenderDrawColor(renderer, 0xFF, 0x00, 0x00, 0xFF);
        break;
    case GREEN:
        SDL_SetRenderDrawColor(renderer, 0x00, 0xFF, 0x00, 0xFF);
        break;
    case BLUE:
        SDL_SetRenderDrawColor(renderer, 0x00, 0x00, 0xFF, 0xFF);
        break;
    case YELLOW:
        SDL_SetRenderDrawColor(renderer, 0xFF, 0xFF, 0x00, 0xFF);
        break;
    case MAGENTA:
        SDL_SetRenderDrawColor(renderer, 0xFF, 0x00, 0xFF, 0xFF);
        break;
    case CYAN:
        SDL_SetRenderDrawColor(renderer, 0x00, 0xFF, 0xFF, 0xFF);
        break;
    case GRAY:
        SDL_SetRenderDrawColor(renderer, 0x80, 0x80, 0x80, 0xFF);
        break;
    case ORANGE:
        SDL_SetRenderDrawColor(renderer, 0xFF, 0xA5, 0x00, 0xFF);
        break;
    case PURPLE:
        SDL_SetRenderDrawColor(renderer, 0x80, 0x00, 0x80, 0xFF);
        break;
    default:
        break;
}


    SDL_RenderFillRect(renderer, &(SDL_Rect){ x + 1, y + 1, 18, 18 });
}


void PrintMatrix(CellStatus matrix[MATRIX_HEIGHT][MATRIX_WIDTH]) {
    for (int y = 0; y < MATRIX_HEIGHT; y++) {
        for (int x = 0; x < MATRIX_WIDTH; x++) {
            printf("%d ", matrix[y][x]);
        }
        printf("\n");
    }
}


CellStatus currentBlock = EMPTY;
CellStatus nextBlock = EMPTY;

void Spawn_block(){
	srand(time(NULL));
	if(currentBlock==EMPTY && nextBlock!=EMPTY){
		currentBlock = nextBlock;
		nextBlock = rand () %8 +2;
	}else{
		currentBlock = rand()%8 +2;
		nextBlock=rand()%8+2;
	}

	for(int i=0;i<4;i++){
		for(int j=0;j<2;j++){
			matrix[2+j][14+i] = blocks[currentBlock][j][i];
		}
	}
	for(int i=0;i<4;i++){
		for(int j=0;j<2;j++){
			matrix[4+j][2+i] = blocks[nextBlock][j][i];
		}
	}


	
}

void draw_game_board(){
    for (int x = 0; x < MATRIX_HEIGHT; x++) {
        for (int y = 0; y < MATRIX_WIDTH; y++) {
		if(matrix[x][y]==FRAME){
			Draw_rect(GRAY,x*20,y*20);
		}
		else if(matrix[x][y]==BLOCK_SQUARE){
			Draw_rect(RED,x*20,y*20);
		}
		else if(matrix[x][y]==BLOCK_LINE){
			Draw_rect(GREEN,x*20,y*20);
		}
		else if(matrix[x][y]==BLOCK_T){
			Draw_rect(BLUE,x*20,y*20);
		}
		else if(matrix[x][y]==BLOCK_L){
			Draw_rect(YELLOW,x*20,y*20);
		}
		else if(matrix[x][y]==BLOCK_L_REV){
			Draw_rect(MAGENTA,x*20,y*20);
		}
		else if(matrix[x][y]==BLOCK_ZIGZAG){
			Draw_rect(CYAN,x*20,y*20);
		}
		else if(matrix[x][y]==BLOCK_ZIGZAG_REV){
			Draw_rect(ORANGE,x*20,y*20);
		}
	}
    }
    
}

void draw_text(int x,int y,char* str,SDL_Color textColor){

    SDL_Surface *textSurface = TTF_RenderText_Solid(font, str, textColor);
    SDL_Texture *textTexture = SDL_CreateTextureFromSurface(renderer, textSurface);
    SDL_Rect textRect = { x, y, textSurface->w, textSurface->h };
    SDL_RenderCopy(renderer, textTexture, NULL, &textRect);
}

```





